## CaNH2 structure including vibronic, rotational and electronic spin

In [1]:
from src.group_theory.Group import *
from src.molecular_structure.VibronicStates import VibronicState, OffsetOperator
from src.molecular_structure.RotationalStates import STM_RotationalBasis
from src.molecular_structure.molecular_Hamiltonians.SpinRotation import Spin_Rotation_Hamiltonian_evCaseB
from src.quantum_mechanics.AngularMomentum import ElectronicSpinBasis
from src.quantum_mechanics.Basis import OrthogonalBasis
from src.molecular_structure.molecular_Hamiltonians.Rotational_Hamiltonian import *

# uncomment the following if you want to print matrices and not see all the digits.
# np.set_printoptions(formatter={'all': lambda x: f"{x.real:.2f}"},threshold=np.inf)


In [2]:
def wavenumber_to_Hz(k):
    if isinstance(k, list):
        return [wavenumber_to_Hz(item) for item in k]
    if isinstance(k, dict):
        return {key: wavenumber_to_Hz(k[key]) for key in k}
    return k * 299792458 * 100

def wavenumber_to_GHz(k):
    if isinstance(k, list):
        return [wavenumber_to_GHz(item) for item in k]
    if isinstance(k, dict):
        return {key: wavenumber_to_GHz(k[key]) for key in k}
    return k * 299792458 * 100 / 1e9

# define symmetry group
group = C2vGroup()
A1 = C2v_A1_representation(group)
A2 = C2v_A2_representation(group)
B1 = C2v_B1_representation(group)
B2 = C2v_B2_representation(group)

### Define the vibronic basis, and their dipole moment.

Here I only included one vibrational state in each electronic state. Just add more if needed

In [3]:
from src.tools.SphericalTensors import SphericalTensor_prolate

X = VibronicState("X", symmetry_group=group, irrep= A1)
A = VibronicState("A", symmetry_group=group, irrep= B2)
B = VibronicState("B", symmetry_group=group, irrep= B1)
vibronic_basis = OrthogonalBasis([X,A,B],"Electronic")
vibronic_d_a = Operator(vibronic_basis,np.array([[0,0,0],[0,0,0],[0,0,0]]),symmetry_group=group, irrep= A1)
vibronic_d_b = Operator(vibronic_basis,np.array([[0,1,0],[1,0,0],[0,0,0]]),symmetry_group=group, irrep= B2)
vibronic_d_c = Operator(vibronic_basis,np.array([[0,0,1],[0,0,0],[1,0,0]]),symmetry_group=group, irrep= B1)

# the transition dipole moment in molecule frame as a spherical tensor
vibronic_d = SphericalTensor_prolate(np.array([vibronic_d_a,vibronic_d_b,vibronic_d_c]),is_operator=True, operator_basis=vibronic_basis)

### Define the rotational basis and electron spin basis

Note: it's important to couple vibronic and rotational states first, before coupling to other states. This is because the dipole moment operator evaluation is not separable for vibronic and rotational Hilbert spaces.

In [7]:
rot_basis = STM_RotationalBasis(R_range=(0,2))
es_basis = ElectronicSpinBasis(S_range=(1/2,1/2))
evr_basis = rot_basis * vibronic_basis

basis = evr_basis.couple(es_basis)
basis.rename_symbols("J","m") # rename angular momentum symbols.

### Define molecular constants for different electronic states

Note: all constants should be given in GHz.

In [8]:
from src.molecular_structure.RotationOperators import MShiftOperator, JShiftOperator

A_dict = {}
BC_avg2_dict = {}
BC_diff4_dict = {}

e_aa_dict = {}
e_bb_dict = {}
e_cc_dict = {}

offset_dict = {}

# X state constants
A_dict["X"] = wavenumber_to_GHz(13.05744)
BC_avg2_dict["X"] = wavenumber_to_GHz(0.296652)
BC_diff4_dict["X"] = wavenumber_to_GHz(1.8894e-3)
e_aa_dict["X"] = 45.7e-3
e_bb_dict["X"] = 32.063e-3
e_cc_dict["X"] = 41.110e-3
offset_dict["X"] = 0.0


# A state constants
A_dict["A"] = wavenumber_to_GHz(11.44854)
BC_avg2_dict["A"] = wavenumber_to_GHz(0.303107)
BC_diff4_dict["A"] = wavenumber_to_GHz(1.958e-3)
e_aa_dict["A"] = wavenumber_to_GHz(8.2369)
e_bb_dict["A"] = wavenumber_to_GHz(3.0534e-2  - 1.2617e-2 * 2)
e_cc_dict["A"] = wavenumber_to_GHz(3.0534e-2  + 1.2617e-2 * 2)
offset_dict["A"] = (wavenumber_to_Hz(15464.36739)-192e6)/1e9

# B state constants
A_dict["B"] = wavenumber_to_GHz(14.3664)
BC_avg2_dict["B"] = wavenumber_to_GHz(0.301693)
BC_diff4_dict["B"] = wavenumber_to_GHz(4.68e-3)
e_aa_dict["B"] = wavenumber_to_GHz(-7.5472)
e_bb_dict["B"] = wavenumber_to_GHz(2.083e-2 - 8.66e-3 * 2)
e_cc_dict["B"] = wavenumber_to_GHz(2.083e-2 + 8.66e-3 * 2)
offset_dict["B"] = (wavenumber_to_Hz(15885.28188)-129e6)/1e9

### Define the Hamiltonian

H_rot: rotational Hamiltonian

H_SR: spin-rotation coupling

H_offset: adds offset to the energies for different electronic states

Z: breaks magnetic degeneracies by adding a fictitious term

In [9]:
H_rot = Rotational_Hamiltonian_evr(basis,A_dict,BC_avg2_dict,BC_diff4_dict)
H_SR = Spin_Rotation_Hamiltonian_evCaseB(basis, e_aa_dict, e_bb_dict, e_cc_dict)
H_offset = OffsetOperator(basis,offset_dict)
Z = MShiftOperator(basis) * 1e-5 #+ JShiftOperator(basis) * 1e-5


H = H_rot + H_SR + H_offset + Z

In [10]:
# you can check which basis states are coupled by each operator by calling
# H_SR.get_connected_states()

### Diagonalize the Hamiltonian

In [11]:
from src.molecular_structure.RotationalStates import rename_ATM_states

Es, states = H.diagonalize()
rename_ATM_states(states)  # rename the states using ATM convention (N ka kc)

### Inspect the energy eigenstates

We can take a look at some of the eigenstates. For simplicity let's restrict to the m = 1/2 magnetic states

### X State

In [12]:
X_states_m12 = []
X_Es_m12 = []
for i in range(len(states)):
    if states[i][0].m == 1/2 and states[i][0].elec == "X":
        X_states_m12.append(states[i])
        X_Es_m12.append(Es[i])

These are our cycling ground states:

In [13]:
X_states_m12[5]

|elec = X, S = 0.5, N=1, ka=1, kc=1, J = 0.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=1, k=1, J=0.5, m=0.5> + -0.71 |elec=X, S=0.5, N=1, k=-1, J=0.5, m=0.5> 

In [14]:
X_states_m12[6]

|elec = X, S = 0.5, N=1, ka=1, kc=1, J = 1.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=1, k=1, J=1.5, m=0.5> + -0.71 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=0.5> + 3.9e-05 |elec=X, S=0.5, N=2, k=-1, J=1.5, m=0.5> + 3.9e-05 |elec=X, S=0.5, N=2, k=1, J=1.5, m=0.5> 

In [15]:
print(f"1_11 state spin-rotation splitting = {(X_Es_m12[6] - X_Es_m12[5])*1000:.1f} MHz")

1_11 state spin-rotation splitting = 65.1 MHz


These are the 2_11 states:

In [16]:
X_states_m12[11]

|elec = X, S = 0.5, N=2, ka=1, kc=1, J = 1.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=2, k=-1, J=1.5, m=0.5> + 0.71 |elec=X, S=0.5, N=2, k=1, J=1.5, m=0.5> + -3.9e-05 |elec=X, S=0.5, N=1, k=1, J=1.5, m=0.5> + 3.9e-05 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=0.5> 

In [17]:
X_states_m12[12]

|elec = X, S = 0.5, N=2, ka=1, kc=1, J = 2.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=2, k=-1, J=2.5, m=0.5> + 0.71 |elec=X, S=0.5, N=2, k=1, J=2.5, m=0.5> 

In [18]:
print(f"2_11 state spin-rotation splitting = {(X_Es_m12[12] - X_Es_m12[11])*1000:.1f} MHz")

2_11 state spin-rotation splitting = 89.6 MHz


### Optional: decompose case B into STM basis

If you are curious about the decomposition into rotation x spin, you can do it like this:

In [19]:
STM_basis = basis.get_uncoupled_basis()
matrix = basis.get_basis_change_matrix()
s = X_states_m12[5].change_basis(STM_basis,matrix)
s

|elec = X, S = 0.5, N=1, ka=1, kc=1, J = 0.5, m = 0.5> = 0.58 |elec=X, S=0.5, ms=-0.5, N=1, k=1, mN=1> + -0.58 |elec=X, S=0.5, ms=-0.5, N=1, k=-1, mN=1> + -0.41 |elec=X, S=0.5, ms=0.5, N=1, k=1, mN=0> + 0.41 |elec=X, S=0.5, ms=0.5, N=1, k=-1, mN=0> 

### A State

In [20]:
A_states_m12 = []
A_Es_m12 = []
for i in range(len(states)):
    if states[i][0].m == 1/2 and states[i][0].elec == "A":
        A_states_m12.append(states[i])
        A_Es_m12.append(Es[i])

This is the A 0_00 J=1/2 state, which is the excited state in our cycling scheme:

In [21]:
A_states_m12[0]

|elec = A, S = 0.5, N=0, ka=0, kc=0, J = 0.5, m = 0.5> = 1.00 |elec=A, S=0.5, N=0, k=0, J=0.5, m=0.5> 

In [22]:
print(f"Cycling transition frequency on X-A: {(A_Es_m12[0] - X_Es_m12[5])*1e-3:.6f} THz")

Cycling transition frequency on X-A: 463.209690 THz


### B State

In [23]:
B_states_m12 = []
B_Es_m12 = []
for i in range(len(states)):
    if states[i][0].m == 1/2 and states[i][0].elec == "B":
        B_states_m12.append(states[i])
        B_Es_m12.append(Es[i])

This is the B 0_00 state, which is the excited state for X-B cycling scheme:

In [24]:
B_states_m12[0]

|elec = B, S = 0.5, N=0, ka=0, kc=0, J = 0.5, m = 0.5> = 1.00 |elec=B, S=0.5, N=0, k=0, J=0.5, m=0.5> 

For completeness, these are the ground states for the X-B cycling scheme:

In [25]:
X_states_m12[7]

|elec = X, S = 0.5, N=1, ka=1, kc=0, J = 0.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=1, k=-1, J=0.5, m=0.5> + 0.71 |elec=X, S=0.5, N=1, k=1, J=0.5, m=0.5> 

In [26]:
X_states_m12[8]

|elec = X, S = 0.5, N=1, ka=1, kc=0, J = 1.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=0.5> + 0.71 |elec=X, S=0.5, N=1, k=1, J=1.5, m=0.5> + -1.2e-04 |elec=X, S=0.5, N=2, k=-1, J=1.5, m=0.5> + 1.2e-04 |elec=X, S=0.5, N=2, k=1, J=1.5, m=0.5> 

In [27]:
print(f"Cycling transition frequency on X-B: {(B_Es_m12[0] - X_Es_m12[7])*1e-3:.6f} THz")

Cycling transition frequency on X-B: 475.828221 THz


Let's also look at the B 1_01 SR states (excited states for our 6_2 repump):

In [28]:
B_states_m12[1]

|elec = B, S = 0.5, N=1, ka=0, kc=1, J = 0.5, m = 0.5> = 1.00 |elec=B, S=0.5, N=1, k=0, J=0.5, m=0.5> 

In [29]:
B_states_m12[2]

|elec = B, S = 0.5, N=1, ka=0, kc=1, J = 1.5, m = 0.5> = 1.00 |elec=B, S=0.5, N=1, k=0, J=1.5, m=0.5> + 1.6e-04 |elec=B, S=0.5, N=2, k=2, J=1.5, m=0.5> + -1.6e-04 |elec=B, S=0.5, N=2, k=-2, J=1.5, m=0.5> 

In [30]:
print(f"Spin-rotation on B 1_01: {(B_Es_m12[2] - B_Es_m12[1]):.6f} GHz")

Spin-rotation on B 1_01: 0.936598 GHz


## Transition Dipole Moments

Define the dipole operator on the vibronic x rotational Hilbert space and on the electron spin Hilbert space. Take their tensor product.

sigma_d is the spatial angular momentum of the dipole moment. So sigma_d = 0 is looking at the pi transitions.

In [31]:
from src.molecular_structure.TDMs import DipoleOperator_evr, DipoleOperator_spin

sigma_d = 0
d_evr = DipoleOperator_evr(evr_basis,vibronic_d, 0)
d_es = DipoleOperator_spin(es_basis,0)
d_uncoupled = d_evr.tensor(d_es)
d = d_uncoupled.change_basis(basis, basis.get_basis_change_matrix())

For example, check the dipole moment between X 1_11 J=3/2, m=1/2 and A 0_00 J=1/2, m=1/2:

In [44]:
s1 = X_states_m12[6]
s2 = A_states_m12[0]
print(f"{d.matrix_element(s1, s2):.2f}")

-0.47-0.00j


And check the dipole moment between X 1_10 J=3/2, m=1/2 and A 0_00 J=1/2, m=1/2: (it should be zero, because X 1_01 has the same parity as A 0_00)

In [33]:
s1 = X_states_m12[8]
s2 = A_states_m12[0]
print(f"{d.matrix_element(s1, s2):.2f}")

0.00+0.00j


It is, instead, connected to B 0_00 state:

In [42]:
s1 = X_states_m12[8]
s2 = B_states_m12[0]
print(f"{d.matrix_element(s1, s2):.2f}")

-0.00-0.47j


Look at dipole-connected Case B basis states using d.get_connected_states()

In [45]:
connected_states = d.get_connected_states()
connected_states

[(|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=0.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=1.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=B, S=0.5, N=1, k=-1, J=0.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=B, S=0.5, N=1, k=-1, J=1.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=1, J=0.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=1, J=1.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=B, S=0.5, N=1, k=1, J=0.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=B, S=0.5, N=1, k=1, J=1.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=0.5, m=0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=1.5, m=0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=0.5>,
  |elec=B, S=0.5, N=1, k=-1, J=0.5, m=0.5>),
 (|elec=X, S=0.5, N=